In [18]:
import nltk; 
import time
import os
import re
import numpy as np
import pandas as pd
from pprint import pprint
from string import digits
from string import punctuation
import unidecode
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [19]:
from gensim.models.wrappers.ldamallet import malletmodel2ldamodel

In [20]:
LdaModel = gensim.models.ldamodel.LdaModel





In [21]:
region = 'europe'

In [22]:
#load model
ldamodel= LdaModel.load("../data/cambridge_analytica/regional_datasets/files_"+region+"/english_"+region+"_tweets_20190411.csv_gensim.model")

In [23]:
#corpus
with open('../data/cambridge_analytica/regional_datasets/files_'+region+'/english_'+region+'_tweets_20190411.csv_corpus.pkl', 'rb') as f:
    corpus = pickle.load(f)


In [24]:
#load data
file_name = 'english_'+region+'_tweets_20190411.csv'
#file_name = 'english_europe_tweets_20190411.csv'


df  = pd.read_csv('../data/cambridge_analytica/regional_datasets/'+file_name)

In [25]:
matrix_documents_topic_contribution= pd.DataFrame()
for i, row in enumerate(ldamodel[corpus]): #i = index_row, row = (topic_index, contribution)
    matrix_documents_topic_contribution = matrix_documents_topic_contribution.append(pd.DataFrame(pd.DataFrame(row).T.iloc[1,:]).T, ignore_index=True)
    if i%10000 == 0:
        print(i)

0
10000
20000
30000
40000
50000
60000
70000


KeyboardInterrupt: 

In [61]:
# Add original text to the end of the output
contents = pd.Series(df['text'])
matrix_documents_topic_contribution = pd.concat([matrix_documents_topic_contribution, contents], axis=1)

In [62]:
# Add original text to the end of the output
contents = pd.Series(df['full_text'])
matrix_documents_topic_contribution = pd.concat([matrix_documents_topic_contribution, contents], axis=1)

In [63]:
# Add original text to the end of the output
contents = pd.Series(df['texto_completo'])
matrix_documents_topic_contribution = pd.concat([matrix_documents_topic_contribution, contents], axis=1)

In [64]:
matrix_documents_topic_contribution = matrix_documents_topic_contribution.reset_index()


In [65]:
matrix_documents_topic_contribution.head()

,index,0,1,2,3,4,5,6,7,8,9,10,text,full_text,texto_completo
0,0,0.079757,0.079745,0.079745,0.132669,0.097288,0.079757,0.079757,0.079745,0.097289,0.114503,0.079745,@SenTedCruz Hold Facebook accountable for abus...,NaN,<usernameremoved> Hold Facebook accountable fo...
1,1,0.082645,0.100826,0.082645,0.098908,0.082645,0.095298,0.088173,0.082645,0.102745,0.100826,0.082645,Both sides should just drop it. Trump won...el...,NaN,Both sides should just drop it. Trump won...el...
2,2,0.110938,0.093992,0.077042,0.087434,0.110940,0.077284,0.084232,0.093113,0.093991,0.077042,0.093993,$AAPL lobbing threes while they can. Wasnt it ...,$AAPL lobbing threes while they can. Wasnt it ...,$AAPL lobbing threes while they can. Wasnt it ...
3,3,0.085763,0.085763,0.142317,0.085763,0.085763,0.085763,0.085763,0.085763,0.085763,0.085813,0.085763,15 min #RSI Signals:\n\n$BTC - $SLR: 6.84\n$BT...,15 min #RSI Signals:\n\n$BTC - $SLR: 6.84\n$BT...,15 min #RSI Signals:\n\n$BTC - $SLR: 6.84\n$BT...
4,4,0.079757,0.079745,0.079745,0.132669,0.097288,0.079757,0.079757,0.079745,0.097289,0.114503,0.079745,@SenJohnThune Hold Facebook accountable for ab...,NaN,<usernameremoved> Hold Facebook accountable fo...


In [66]:
matrix_documents_topic_contribution.to_csv(file_name+'_matrix_document_topic_distribution.csv')

# Get input - most relevant documents for TopicVisExplorer


In [34]:
#load baseline
import pickle
with open('../data/cambridge_analytica/regional_datasets/files_europe/english_europe_tweets_20190411.csvsent_topics_sorteddf_mallet_ldamodel', 'rb') as f:
    baseline =  pickle.load(f)

In [35]:
baseline

,Topic_Num,Topic_Perc_Contrib,Keywords,clean_text,Unnamed: 0,created_at2,id,created_at,user.screen_name,user_location,coordinates,text,full_text,retweet_created_at,retweet_full_text,texto_completo,ubicacion_encontrada
0,0.0,0.2802,"people, delete, year, account, friend, time, b...",\n nycacc\n dogsoftwitter\nrocky\n attention\...,559512,2018-04-30,id-991089497155821568,Mon Apr 30 22:58:44 +0000 2018,MIMI_A1067931,GERMANY,NaN,.\n#NYCACC\n#DogsOfTwitter\nROCKY\n#ATTENTION\...,.\n#NYCACC\n#DogsOfTwitter\nROCKY\n#ATTENTION\...,NaN,NaN,.\n#NYCACC\n#DogsOfTwitter\nROCKY\n#ATTENTION\...,Germany
1,0.0,0.2776,"people, delete, year, account, friend, time, b...",nycacc\n dogsoftwitter\ncapone vid\n attenti...,565365,2018-04-30,id-990774820777406464,Mon Apr 30 02:08:19 +0000 2018,MIMI_A1067931,GERMANY,NaN,#NYCACC\n#DogsOfTwitter\nCAPONE #VID\n#ATTENTI...,#NYCACC\n#DogsOfTwitter\nCAPONE #VID\n#ATTENTI...,NaN,NaN,#NYCACC\n#DogsOfTwitter\nCAPONE #VID\n#ATTENTI...,Germany
2,0.0,0.2730,"people, delete, year, account, friend, time, b...",usernameremoved i can t wait for the followi...,377905,2018-04-13,id-984906143129620480,Fri Apr 13 21:28:17 +0000 2018,StarkieJake,"England, United Kingdom",NaN,<usernameremoved> I can't wait for the followi...,@DurvTM I can't wait for the following :\nCall...,NaN,NaN,<usernameremoved> I can't wait for the followi...,United Kingdom
3,0.0,0.2689,"people, delete, year, account, friend, time, b...",\n nycacc\n dogsoftwitter\natlas vid\n atten...,559426,2018-04-30,id-991090010781880321,Mon Apr 30 23:00:46 +0000 2018,MIMI_A1067931,GERMANY,NaN,.\n#NYCACC\n#DogsOfTwitter\nATLAS #VID\n#ATTEN...,.\n#NYCACC\n#DogsOfTwitter\nATLAS #VID\n#ATTEN...,NaN,NaN,.\n#NYCACC\n#DogsOfTwitter\nATLAS #VID\n#ATTEN...,Germany
4,0.0,0.2672,"people, delete, year, account, friend, time, b...",apparently all my friends are talking about me...,773686,2018-06-01,id-1002645439836016640,Fri Jun 01 20:17:55 +0000 2018,WendyLatto,Scotland,NaN,Apparently all my friends are talking about me...,Apparently all my friends are talking about me...,NaN,NaN,Apparently all my friends are talking about me...,United Kingdom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1241,10.0,0.1736,"good, read, make, great, interesting, real, ar...",facebook recommended for you studiovibesturn...,211010,2018-04-10,id-983601776917123073,Tue Apr 10 07:05:12 +0000 2018,svtup_join,"England, United Kingdom",NaN,#facebook recommended for you #StudioVibesTurn...,NaN,NaN,NaN,#facebook recommended for you #StudioVibesTurn...,United Kingdom
1242,10.0,0.1736,"good, read, make, great, interesting, real, ar...",facebook recommended for you studiovibesturn...,212857,2018-04-10,id-983631977969520640,Tue Apr 10 09:05:13 +0000 2018,svtup_join,"England, United Kingdom",NaN,#facebook recommended for you #StudioVibesTurn...,NaN,NaN,NaN,#facebook recommended for you #StudioVibesTurn...,United Kingdom
1243,10.0,0.1736,"good, read, make, great, interesting, real, ar...",facebook recommended for you studiovibesturn...,214820,2018-04-10,id-983556477318258688,Tue Apr 10 04:05:12 +0000 2018,svtup_join,"England, United Kingdom",NaN,#facebook recommended for you #StudioVibesTurn...,NaN,NaN,NaN,#facebook recommended for you #StudioVibesTurn...,United Kingdom
1244,10.0,0.1736,"good, read, make, great, interesting, real, ar...",facebook recommended for you studiovibesturn...,215153,2018-04-10,id-983586676088942592,Tue Apr 10 06:05:12 +0000 2018,svtup_join,"England, United Kingdom",NaN,#facebook recommended for you #StudioVibesTurn...,NaN,NaN,NaN,#facebook recommended for you #StudioVibesTurn...,United Kingdom


In [27]:
name = 'english_europe_tweets_20190411.csv_matrix_document_topic_distribution.csv'
region = 'europe'

In [29]:
df = pd.read_csv("../data/cambridge_analytica/regional_datasets/files_"+region+"/"+name)

In [30]:
df

,Unnamed: 0,index,0,1,2,3,4,5,6,7,8,9,10,text,full_text,texto_completo
0,0,0,0.067683,0.075224,0.111145,0.083138,0.087522,0.095556,0.066845,0.120091,0.133339,0.066889,0.092569,Meet our newest product: the transparent CamHa...,Meet our newest product: the transparent CamHa...,Meet our newest product: the transparent CamHa...
1,1,1,0.095597,0.098896,0.078370,0.078370,0.078370,0.112853,0.078630,0.078384,0.125669,0.079250,0.095611,@SupportOurLefty This week has been the most #...,@SupportOurLefty This week has been the most #...,<usernameremoved> This week has been the most ...
2,2,2,0.087413,0.087413,0.087413,0.087643,0.087413,0.087413,0.087413,0.106643,0.106413,0.087413,0.087413,Pulitzer prize for this lady. Investigative jo...,NaN,Pulitzer prize for this lady. Investigative jo...
3,3,3,0.083831,0.092424,0.136948,0.097352,0.075758,0.109086,0.077989,0.089814,0.075758,0.076336,0.084705,"Theres no more secrecy, confident and privacy ...","Theres no more secrecy, confident and privacy ...","Theres no more secrecy, confident and privacy ..."
4,4,4,0.081169,0.148244,0.096213,0.093189,0.081169,0.081169,0.093545,0.081761,0.081204,0.081169,0.081169,AggregateIQ: the obscure Canadian tech firm an...,NaN,AggregateIQ: the obscure Canadian tech firm an...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111740,111740,111740,0.081820,0.072192,0.078380,0.178688,0.072150,0.072177,0.119700,0.088103,0.072150,0.092488,0.072150,Calling all #startups that can bring value to ...,Calling all #startups that can bring value to ...,Calling all #startups that can bring value to ...
111741,111741,111741,0.156446,0.078370,0.080614,0.080978,0.078370,0.078383,0.078370,0.078370,0.109758,0.098949,0.081393,@pjmears @OwenJones84 @McrMomentum @PplsAssemb...,@pjmears @OwenJones84 @McrMomentum @PplsAssemb...,<usernameremoved> <usernameremoved> <usernamer...
111742,111742,111742,0.167842,0.079745,0.079745,0.079745,0.096877,0.079745,0.079779,0.086952,0.079745,0.080613,0.089214,Nothing makes me more upset than when Facebook...,NaN,Nothing makes me more upset than when Facebook...
111743,111743,111743,0.089127,0.089127,0.089127,0.089127,0.089127,0.089127,0.089127,0.089127,0.108734,0.089127,0.089127,Do you use hashtags in your Facebook ads? This...,NaN,Do you use hashtags in your Facebook ads? This...
